# Implementation
<div id="wave:2D3D:impl"></div>


We shall now describe in detail various Python implementations
for solving a standard 2D, linear wave equation with constant
wave velocity and $u=0$ on the
boundary. The wave equation is to be solved
in the space-time domain $\Omega\times (0,T]$,
where $\Omega = (0,L_x)\times (0,L_y)$ is a rectangular spatial
domain. More precisely,
the complete initial-boundary value problem is defined by

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
u_{tt} = c^2(u_{xx} + u_{yy}) + f(x,y,t),\quad (x,y)\in \Omega,\ t\in (0,T],
\label{_auto1} \tag{1}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation} 
u(x,y,0) = I(x,y),\quad (x,y)\in\Omega,
\label{_auto2} \tag{2}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation} 
u_t(x,y,0) = V(x,y),\quad (x,y)\in\Omega,
\label{_auto3} \tag{3}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation} 
u = 0,\quad (x,y)\in\partial\Omega,\ t\in (0,T],
\label{_auto4} \tag{4}
\end{equation}
$$

where $\partial\Omega$ is the boundary of $\Omega$, in this case
the four sides of the rectangle $\Omega = [0,L_x]\times [0,L_y]$:
$x=0$, $x=L_x$, $y=0$, and $y=L_y$.

The PDE is discretized as

$$
[D_t D_t u = c^2(D_xD_x u + D_yD_y u) + f]^n_{i,j},
$$

which leads to an explicit updating formula to be implemented in a
program:

$$
u^{n+1}_{i,j} = -u^{n-1}_{i,j} + 2u^n_{i,j} + \nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="wave:2D3D:impl1:2Du0:ueq:discrete"></div>

$$
\begin{equation} 
\quad C_x^2(
u^{n}_{i+1,j} - 2u^{n}_{i,j} + u^{n}_{i-1,j}) + C_y^2
(u^{n}_{i,j+1} - 2u^{n}_{i,j} + u^{n}_{i,j-1}) + \Delta t^2 f_{i,j}^n,
\label{wave:2D3D:impl1:2Du0:ueq:discrete} \tag{5}
\end{equation}
$$

for all interior mesh points $i\in\seti{\mathcal{I}_x}$ and
$j\in\seti{\mathcal{I}_y}$, for $n\in\setr{\mathcal{I}_t}$.
The constants $C_x$ and $C_y$ are defined as

$$
C_x = c\frac{\Delta t}{\Delta x},\quad C_y = c\frac{\Delta t}{\Delta y}
\thinspace .
$$

At the boundary, we simply set $u^{n+1}_{i,j}=0$ for
$i=0$, $j=0,\ldots,N_y$; $i=N_x$, $j=0,\ldots,N_y$;
$j=0$, $i=0,\ldots,N_x$; and $j=N_y$, $i=0,\ldots,N_x$.
For the first step, $n=0$, ([5](#wave:2D3D:impl1:2Du0:ueq:discrete))
is combined with the discretization of the initial condition $u_t=V$,
$[D_{2t} u = V]^0_{i,j}$ to obtain a special formula for
$u^1_{i,j}$ at the interior mesh points:

$$
u^{1}_{i,j} = u^0_{i,j} + \Delta t V_{i,j} + \nonumber
$$

$$
\quad {\frac{1}{2}}C_x^2(
u^{0}_{i+1,j} - 2u^{0}_{i,j} + u^{0}_{i-1,j}) + {\frac{1}{2}}C_y^2
(u^{0}_{i,j+1} - 2u^{0}_{i,j} + u^{0}_{i,j-1}) +\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="wave:2D3D:impl1:2Du0:ueq:n0:discrete"></div>

$$
\begin{equation} 
\quad \frac{1}{2}\Delta t^2f_{i,j}^n,
\label{wave:2D3D:impl1:2Du0:ueq:n0:discrete} \tag{6}
\end{equation}
$$

The algorithm is very similar to the one in 1D:

1. Set initial condition $u^0_{i,j}=I(x_i,y_j)$

2. Compute $u^1_{i,j}$ from ([5](#wave:2D3D:impl1:2Du0:ueq:discrete))

3. Set $u^1_{i,j}=0$ for the boundaries $i=0,N_x$, $j=0,N_y$

4. For $n=1,2,\ldots,N_t$:

 a. Find $u^{n+1}_{i,j}$ from ([5](#wave:2D3D:impl1:2Du0:ueq:discrete))
    for all internal mesh points, $i\in\seti{\mathcal{I}_x}$, $j\in\seti{\mathcal{I}_y}$

 b. Set $u^{n+1}_{i,j}=0$ for the boundaries $i=0,N_x$, $j=0,N_y$


<div id="wave2D3D:impl:scalar"></div>


### The wave propagator

The `solver` function for a 2D case with constant wave velocity and
boundary condition $u=0$ is analogous to the 1D case with similar parameter
values (see `wave1D_u0.py`), apart from a few necessary
extensions. The code is found in the program
[`wave2D_u0.py`](${src_wave}/wave2D_u0/wave2D_u0.py).


The spatial domain is now $[0,L_x]\times [0,L_y]$, specified
by the arguments `Lx` and `Ly`. Similarly, the number of mesh
points in the $x$ and $y$ directions,
$N_x$ and $N_y$, become the arguments `Nx` and `Ny`.
In multi-dimensional problems it makes less sense to specify a
Courant number since the wave velocity is a vector and mesh spacings
may differ in the various spatial directions.
We therefore give $\Delta t$ explicitly. The `solver` function is implemented
and explained below.

In [ ]:
import numpy as np
from math import sqrt
from devito import *

def solver(I, V, f_init, c, bc, Lx, Ly, Nx, Ny, dt, T):
    # Define the physical domain
    grid = Grid(shape=(Nx, Ny), extent=(Lx, Ly))

    t = grid.stepping_dim
    time = grid.time_dim
    x, y = grid.dimensions
    dx, dy = grid.spacing
    
    if dt <= 0:                # max time step?
        dt = (1/float(c))*(1/sqrt(1/dx**2 + 1/dy**2))
    Nt = int(round(T/float(dt)))

    cx2 = (c*dt/dx)**2
    cy2 = (c*dt/dy)**2

    # A Function to represent the space-varying field `f`
    f = Function(name='f', grid=grid, initializer=f_init)

    # A TimeFunction to represent the time- and space-varying field `u`
    # Here `u^{n}_{i,j}` corresponds to `u(t, x, y)`
    u = TimeFunction(name='u', grid=grid, space_order=2, save=Nt+1)
    u0 = u.shift(time, -time)
    u1 = u.shift(time, 1 - time)

    # Load initial condition at t=0 into `u0`    
    I(u0)

    # The time marching updating equations
    u1_update = Eq(u1, u0 + dt*V(u0) + 0.5*(cx2*u0.dx2 + cy2*u0.dy2 + dt**2*f))
    un_update = Eq(u.forward, -u.backward + 2*u + cx2*u.dx2 + cy2*u.dy2 + dt**2*f)

    # We create an Operator, which joins together the time marching equation
    # and the BCs
    op = Operator([u1_update, un_update] + bc(u))

    # Finally, we can execute the Operator for the given number of timesteps
    op.apply(time_m=2, time_M=5)
    #op.apply(time_m=2, time_M=Nt)
    
    return u

### Boundary conditions

The following functions build several sorts of boundary conditions as lists of `Eq`'s.

In [ ]:
def make_dirichlet_bc(u):
    Nx, Ny = u.grid.shape
    x, y = u.grid.dimensions
    t = u.grid.stepping_dim

    return [Eq(u[t+1, 0, y], 0),
            Eq(u[t+1, Nx-1, y], 0),
            Eq(u[t+1, x, 0], 0),
            Eq(u[t+1, x, Ny-1], 0)]

In [ ]:
##### TEMPORARY CELL FOR TESTING PURPOSES ######

import os
from glob import glob

from sympy import exp


# Initial Gaussian bell in the middle of the domain.

# Clean up plot files
for name in glob('tmp_*.png'):
    os.remove(name)

Lx = 10
Ly = 10
c = 1.0

def I(u0):
    x, y = u0.function.grid.dimensions
    op = Operator(Eq(u0, exp(-(x-Lx/2.0)**2/2.0 -(y-Ly/2.0)**2/2.0)))
    op.apply()

def f_init(data):
    data[:] = 0.

def V(u0):
    return 0.5*u0

bc = make_dirichlet_bc

Nx = 41
Ny = 41
T = 15
    
u = solver(I, V, f_init, c, bc, Lx, Ly, Nx, Ny, 0, T)

**Remark: How to use the solution.**

The `solver` function in the `wave2D_u0.py` code
updates arrays for the next time step by switching references as
described in the section [wave:pde1:impl:ref:switch](#wave:pde1:impl:ref:switch). Any use of `u` on the
user's side is assumed to take place in the user action function. However,
should the code be changed such that `u` is returned and used as solution,
have in mind that you must return `u_n` after the time limit, otherwise
a `return u` will actually return `u_nm1` (due to the switching of array
indices in the loop)!

## Verification
<div id="wave2D3D:impl:verify"></div>

### Testing a quadratic solution

The 1D solution from the section [wave:pde2:fd:verify:quadratic](#wave:pde2:fd:verify:quadratic) can be
generalized to multi-dimensions and provides a test case where the
exact solution also fulfills the discrete equations, such that we know
(to machine precision) what numbers the solver function should
produce. In 2D we use the following generalization of
([wave:pde2:fd:verify:quadratic:uex](#wave:pde2:fd:verify:quadratic:uex)):

<!-- Equation labels as ordinary links -->
<div id="wave2D3D:impl:verify:quadratic"></div>

$$
\begin{equation}
\uex(x,y,t) = x(L_x-x)y(L_y-y)(1+{\frac{1}{2}}t)
\thinspace .
\label{wave2D3D:impl:verify:quadratic} \tag{7}
\end{equation}
$$

This solution fulfills the PDE problem if $I(x,y)=\uex(x,y,0)$,
$V=\frac{1}{2}\uex(x,y,0)$, and $f=2c^2(1+{\frac{1}{2}}t)(y(L_y-y) +
x(L_x-x))$. To show that $\uex$ also solves the discrete equations,
we start with the general results $[D_t D_t 1]^n=0$, $[D_t D_t t]^n=0$,
and $[D_t D_t t^2]=2$, and use these to compute

$$
\begin{align*}
[D_xD_x \uex]^n_{i,j} &= [y(L_y-y)(1+{\frac{1}{2}}t) D_xD_x x(L_x-x)]^n_{i,j}\\
&= y_j(L_y-y_j)(1+{\frac{1}{2}}t_n)(-2)\thinspace .
\end{align*}
$$

A similar calculation must be carried out for the $[D_yD_y
\uex]^n_{i,j}$ and $[D_tD_t \uex]^n_{i,j}$ terms.  One must also show
that the quadratic solution fits the special formula for
$u^1_{i,j}$. The details are left as [Exercise 1: Check that a solution fulfills the discrete model](#wave:exer:quadratic:2D).
The `test_quadratic` function in the
[`wave2D_u0.py`](${src_wave}/wave2D_u0/wave2D_u0.py)
program implements this verification as a proper test function
for the pytest and nose frameworks.

## Visualization

We want to animate a 3D surface in Matplotlib, but this is a really
slow process and not recommended, so we consider Matplotlib not an
option as long as on-screen animation is desired. One can use the
recipes for single shots of $u$, where it does produce high-quality
3D plots.

The best option for doing visualization of 2D and 3D scalar and vector fields
in Python programs is Mayavi, which is an interface to the high-quality
package VTK in C++. There is good online documentation and also
an introduction in Chapter 5 of [[Langtangen_2012]](#Langtangen_2012).

To obtain Mayavi on Ubuntu platforms you can write

        pip install mayavi --upgrade


For Mac OS X and Windows, we recommend using Anaconda.
To obtain Mayavi for Anaconda you can write

        conda install mayavi


Mayavi has a MATLAB-like interface called `mlab`. We can do

```python
import mayavi.mlab as plt
# or
from mayavi import mlab
```

and have `plt` (as usual) or `mlab`
as a kind of MATLAB visualization access inside our program (just
more powerful and with higher visual quality).

The official documentation of the `mlab` module is provided in two
places, one for the [basic functionality](http://docs.enthought.com/mayavi/mayavi/auto/mlab_helper_functions.html)
and one for [further functionality](http://docs.enthought.com/mayavi/mayavi/auto/mlab_other_functions.html).
Basic [figure
handling](http://docs.enthought.com/mayavi/mayavi/auto/mlab_figure.html)
is very similar to the one we know from Matplotlib.  Just as for
Matplotlib, all plotting commands you do in `mlab` will go into the
same figure, until you manually change to a new figure.

Back to our application, the following code for the user action
function with plotting in Mayavi is relevant to add.

In [ ]:
%matplotlib inline

# Top of the file
try:
    import mayavi.mlab as mlab
except:
    # We don't have mayavi
    pass

# Mayavi visualization
x = np.linspace(0, Lx, Nx+1)
y = np.linspace(0, Ly, Ny+1)

# time consuming!
for t in range(u.shape[0]):
    mlab.clf()
    extent1 = (0, 20, 0, 20,-2, 2)
    s = mlab.surf(x, y, u,
                  colormap='Blues', warp_scale=5, extent=extent1)
    mlab.axes(s, color=(.7, .7, .7), extent=extent1,
              ranges=(0, 10, 0, 10, -1, 1), 
              xlabel='', ylabel='', zlabel='',
              x_axis_visibility=False, z_axis_visibility=False)
    mlab.outline(s, color=(0.7, .7, .7), extent=extent1)
    mlab.text(6, -2.5, '', z=-4, width=0.14)
    mlab.colorbar(object=None, title=None, orientation='horizontal', 
                  nb_labels=None, nb_colors=None, label_fmt=None)
    mlab.title('Gaussian t=%g' % t[n])
    mlab.view(142, -72, 50) 
    f = mlab.gcf()
    camera = f.scene.camera
    camera.yaw(0)

    mlab.savefig(filename)

This is a point to get started - visualization is as always a very
time-consuming and experimental discipline. With the PNG files we
can use `ffmpeg` to create videos.

<!-- dom:FIGURE: [fig-wave/mayavi2D_gaussian1.png, width=600 frac=0.8] Plot with Mayavi. -->
<!-- begin figure -->

<p>Plot with Mayavi.</p>
<img src="fig-wave/mayavi2D_gaussian1.png" width=600>

<!-- end figure -->


<!-- dom:MOVIE: [https://github.com/hplgit/fdm-book/blob/master/doc/pub/book/html/mov-wave/mayavi/wave2D_u0_gaussian/movie.mp4] -->
<!-- begin movie -->

In [ ]:
from IPython.display import HTML
_s = """
<div>
<video  loop controls width='640' height='365' preload='none'>
    <source src='https://github.com/hplgit/fdm-book/blob/master/doc/pub/book/html/mov-wave/mayavi/wave2D_u0_gaussian/movie.mp4'  type='video/mp4;  codecs="avc1.42E01E, mp4a.40.2"'>
    <source src='https://github.com/hplgit/fdm-book/blob/master/doc/pub/book/html/mov-wave/mayavi/wave2D_u0_gaussian/movie.ogg'  type='video/ogg;  codecs="theora, vorbis"'>
</video>
</div>
<p><em></em></p>

<!-- Issue warning if in a Safari browser -->
<script language="javascript">
if (!!(window.safari)) {
  document.write("<div style=\"width: 95%%; padding: 10px; border: 1px solid #100; border-radius: 4px;\"><p><font color=\"red\">The above movie will not play in Safari - use Chrome, Firefox, or Opera.</font></p></div>")}
</script>

"""
HTML(_s)

<!-- end movie -->



# Exercises



<!-- --- begin exercise --- -->

## Exercise 1: Check that a solution fulfills the discrete model
<div id="wave:exer:quadratic:2D"></div>

Carry out all mathematical details to show that
([7](#wave2D3D:impl:verify:quadratic)) is indeed a solution of the
discrete model for a 2D wave equation with $u=0$ on the boundary.
One must check the boundary conditions, the initial conditions,
the general discrete equation at a time level and the special
version of this equation for the first time level.
Filename: `check_quadratic_solution`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Project 2: Calculus with 2D mesh functions
<div id="wave:exer:mesh3D:calculus"></div>

The goal of this project is to redo
[wave:exer:mesh1D:calculus](#wave:exer:mesh1D:calculus) with 2D
mesh functions ($f_{i,j}$).

**Differentiation.**
The differentiation results in a discrete gradient
function, which in the 2D case can be represented by a three-dimensional
array `df[d,i,j]` where `d` represents the direction of
the derivative, and `i,j` is a mesh point in 2D.
Use centered differences for
the derivative at inner points and one-sided forward or backward
differences at the boundary points. Construct unit tests and
write a corresponding test function.

**Integration.**
The integral of a 2D mesh function $f_{i,j}$ is defined as

$$
F_{i,j} = \int_{y_0}^{y_j} \int_{x_0}^{x_i} f(x,y)dxdy,
$$

where $f(x,y)$ is a function that takes on the values of the
discrete mesh function $f_{i,j}$ at the mesh points, but can also
be evaluated in between the mesh points. The particular variation
between mesh points can be taken as bilinear, but this is not
important as we will use a product Trapezoidal rule to approximate
the integral over a cell in the mesh and then we only need to
evaluate $f(x,y)$ at the mesh points.

Suppose $F_{i,j}$ is computed. The calculation of $F_{i+1,j}$
is then

$$
\begin{align*}
F_{i+1,j} &= F_{i,j} + \int_{x_i}^{x_{i+1}}\int_{y_0}^{y_j} f(x,y)dydx\\
& \approx \Delta x \frac{1}{2}\left(
\int_{y_0}^{y_j} f(x_{i},y)dy
+ \int_{y_0}^{y_j} f(x_{i+1},y)dy\right)
\end{align*}
$$

The integrals in the $y$ direction can be approximated by a Trapezoidal
rule. A similar idea can be used to compute $F_{i,j+1}$. Thereafter,
$F_{i+1,j+1}$ can be computed by adding the integral over the final
corner cell to $F_{i+1,j} + F_{i,j+1} - F_{i,j}$. Carry out the
details of these computations and implement a function that can
return $F_{i,j}$ for all mesh indices $i$ and $j$. Use the
fact that the Trapezoidal rule is exact for linear functions and
write a test function.
Filename: `mesh_calculus_2D`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3: Implement Neumann conditions in 2D
<div id="wave:app:exer:wave2D:Neumann"></div>

Modify the [`wave2D_u0.py`](${src_wave}/wave2D_u0/wave2D_u0.py)
program, which solves the 2D wave equation $u_{tt}=c^2(u_{xx}+u_{yy})$
with constant wave velocity $c$ and $u=0$ on the boundary, to have
Neumann boundary conditions: $\partial u/\partial n=0$.
Include both scalar code (for debugging and reference) and
vectorized code (for speed).

To test the code, use $u=1.2$ as solution ($I(x,y)=1.2$, $V=f=0$, and
$c$ arbitrary), which should be exactly reproduced with any mesh
as long as the stability criterion is satisfied.
Another test is to use the plug-shaped pulse
in the `pulse` function from the section [wave:pde2:software](#wave:pde2:software)
and the [`wave1D_dn_vc.py`](${src_wave}/wave1D/wave1D_dn_vc.py)
program. This pulse
is exactly propagated in 1D if $c\Delta t/\Delta x=1$. Check
that also the 2D program can propagate this pulse exactly
in $x$ direction ($c\Delta t/\Delta x=1$, $\Delta y$ arbitrary)
and $y$ direction ($c\Delta t/\Delta y=1$, $\Delta x$ arbitrary).
Filename: `wave2D_dn`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 4: Test the efficiency of compiled loops in 3D
<div id="wave:exer:3D:f77:cy:efficiency"></div>

Extend the `wave2D_u0.py` code and the Cython, Fortran, and C versions to 3D.
Set up an efficiency experiment to determine the relative efficiency of
pure scalar Python code, vectorized code, Cython-compiled loops,
Fortran-compiled loops, and C-compiled loops.
Normalize the CPU time for each mesh by the fastest version.
Filename: `wave3D_u0`.

<!-- --- end exercise --- -->